In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드|
load_dotenv()

True

In [2]:
import os

# LangSmith 추적 비활성화
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_PROJECT"] = ""

In [39]:
from langchain_community.document_loaders import JSONLoader

# JSONLoader를 사용하여 데이터 로드
json_loader = JSONLoader(
    file_path="./data-sample.json",
    jq_schema=".",  # 최상위 리스트의 모든 객체를 로드
    text_content=False  # ✅ page_content가 문자열이 아닐 경우 오류 방지
)

docs = json_loader.load()

# 데이터 확인
print(docs)


[Document(metadata={'source': '/home/chae/RAG/data-sample.json', 'seq_num': 1}, page_content="[{'title': '작가사진', 'title_ch': '作家寫眞', 'title_eng': 'Selfportrait', 'artist': '한기석', 'artist_eng': 'HAN Kisuk', 'artwork_number': 1, 'year': '1960', 'size': '41×51', 'materials': '종이에 젤라틴실버프린트', 'category': '사진', 'description': '‘농(Nong)’이라는 이름으로 미국에서 널리 알려진 한농(韓農) 한기석(1930-2011)은 국내 활동이 그리 많지 않아서 한국 화단에서는 생소한 이름이다. 그가 최초로 한국 화단에 등장한 것은 1971년 11월 신세계 화랑에서 개최한《Nong 展》이후이다. 그는 농(Nong)을 구름 위의 시선(詩仙) 혹은 주선(酒仙)같은 존재로 비유해서 미국에서 자신의 이름으로 쓰고 있다.그의 작품은 전반적으로 자신의 철학적 이미지를 조형화시킨 추상 회화 계통이다. 일종의 형이상학적인 회화 혹은 초현실적인 환상세계라고도 할 수 있는 그의 작품은 양식적인 면에서 주로 구상적인 형태를 취한다.한기석의 <작가사진>(1960)은 본인의 얼굴을 찍은 것으로, 사진 속에서 작가는 자신의 작품을 배경으로 화면의 우측을 주시하고 있다.', 'read_count': 10468}, {'title': '팔괘호', 'title_ch': '八卦壺', 'title_eng': 'Palgwae Vase', 'artist': '한기석', 'artist_eng': 'HAN Kisuk', 'artwork_number': 2, 'year': '1960', 'size': '250×127', 'materials': '캔버스, 종이에 유화 물감', 'category': '회화 II', 'description': '한농(韓農) 한기석(1930-20

In [40]:
docs[0].metadata

{'source': '/home/chae/RAG/data-sample.json', 'seq_num': 1}

In [41]:
# metadata 설정(filename 이 존재해야 함)
for doc in docs:
    doc.metadata["filename"] = doc.metadata["source"]

In [6]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context, conditional
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset.extractor import KeyphraseExtractor
from ragas.testset.docstore import InMemoryDocumentStore

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 데이터셋 생성기
generator_llm = ChatOpenAI(model="gpt-4o")
# 데이터셋 비평기
critic_llm = ChatOpenAI(model="gpt-4o")
# 문서 임베딩
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

/home/chae/faiss_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# 텍스트 분할기를 설정합니다.
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# LangChain의 ChatOpenAI 모델을 LangchainLLMWrapper로 감싸 Ragas와 호환되게 만듭니다.
langchain_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))

# 주요 구문 추출기를 초기화합니다. 위에서 정의한 LLM을 사용합니다.
keyphrase_extractor = KeyphraseExtractor(llm=langchain_llm)

# ragas_embeddings 생성
ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)

# InMemoryDocumentStore를 초기화합니다.
# 이는 문서를 메모리에 저장하고 관리하는 저장소입니다.
docstore = InMemoryDocumentStore(
    splitter=splitter,
    embeddings=ragas_embeddings,
    extractor=keyphrase_extractor,
)

In [8]:
generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    ragas_embeddings,
    docstore=docstore,
)

In [9]:
# 질문 유형별 분포 결정
# simple: 간단한 질문, reasoning: 추론이 필요한 질문, multi_context: 여러 맥락을 고려해야 하는 질문, conditional: 조건부 질문
distributions = {simple: 1}

In [42]:
# 테스트셋 생성
# docs: 문서 데이터, 10: 생성할 질문의 수, distributions: 질문 유형별 분포, with_debugging_logs: 디버깅 로그 출력 여부
testset = generator.generate_with_langchain_docs(
    documents=docs,
    test_size=10,
    distributions=distributions,
    with_debugging_logs=True,
    raise_exceptions=False,
)

embedding nodes:  50%|█████     | 6/12 [00:00<00:00, 11.36it/s][ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['HAN Kisuk', '1960s artwork', 'Palgwae series', 'Korean artist', 'Artistic patterns']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['Selfportrait', 'HAN Kisuk', 'Nong exhibition', 'Korean artist', 'Artistic style']}
embedding nodes:  67%|██████▋   | 8/12 [00:01<00:00,  4.99it/s][ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['HAN Kisuk', '1960s artwork', 'Philosophical imagery', 'Bright background', 'Negative space patterns']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['HAN Kisuk', 'Korean artist', 'Cultural synthesis', 'Philosophical themes', 'Distinctive style']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['HAN Kisuk', '1960s artwork', 'Korean artist', 'Palgwae', 'The Moon and Two Cats']}
Generating:   0%|          | 0/10 [00:00<?, ?it/s]              [ragas.testset.filters.DEBUG] context scoring: {'clarity': 1, 'depth': 2,

In [56]:
# 생성된 테스트셋을 pandas DataFrame으로 변환
df = testset.to_pandas()
df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,\uc6d0\uc2dc\ubbf8\uc220\uc758 \uc601\ud5a5\uc...,"[주제와 색조의 장밋빛 시대(1904–1906)를 지나, 폴 세잔(Paul Céza...",The answer to given question is not present in...,simple,"[{'source': './dataset-sample-1-5.pdf', 'file_...",True
1,In what ways do HAN Kisuk's artworks from the ...,[한기석(1930-2011)은 한국의 역사적 원류 속에서 의식적인 뿌리를 찾고 과거...,HAN Kisuk's artworks from the 1960s incorporat...,simple,"[{'source': '/home/chae/RAG/data-sample.json',...",True
2,What diverse subjects did Pablo Picasso explor...,[1882-1963)와 공동 창시한 큐비즘(1909–1919)을 통해 20세기의 새...,Pablo Picasso explored diverse subjects in his...,simple,"[{'source': './dataset-sample-1-5.pdf', 'file_...",True
3,What techniques did Pablo Picasso use in his a...,"[• 작품 설명: 파블로 피카소(Pablo Picasso, 1881–1973)는 스...",Pablo Picasso used a variety of techniques in ...,simple,"[{'source': './dataset-sample-1-5.pdf', 'file_...",True
4,What distinctive style does Han Kisuk use in h...,[한기석(1930-2011)은 한국의 역사적 원류 속에서 의식적인 뿌리를 찾고 과거...,Han Kisuk uses a distinctive style in his artw...,simple,"[{'source': '/home/chae/RAG/data-sample.json',...",True
5,How does Han Kisuk's work reflect cultural syn...,[한기석(1930-2011)은 한국의 역사적 원류 속에서 의식적인 뿌리를 찾고 과거...,Han Kisuk's work reflects cultural synthesis t...,simple,"[{'source': '/home/chae/RAG/data-sample.json',...",True
6,Who is the Korean artist known for creating th...,"['category': '회화 II', 'description': '한농(韓農) 한...",The Korean artist known for creating the artwo...,simple,"[{'source': '/home/chae/RAG/data-sample.json',...",True
7,Who is the Korean artist responsible for creat...,"['1970', 'size': '75×61', 'materials': '캔버스, 종...",The Korean artist responsible for creating the...,simple,"[{'source': '/home/chae/RAG/data-sample.json',...",True
8,How did Pablo Picasso use color and texture in...,"[• 작품 설명: 파블로 피카소(Pablo Picasso, 1881–1973)는 스...",Pablo Picasso used color and texture in his ar...,simple,"[{'source': './dataset-sample-1-5.pdf', 'file_...",True
9,What elements contribute to the distinctive st...,[표면 묘사에 많은 관심을 가진 작가이다.그는 모나고 약간 무게가 있는 듯이 보이는...,Han Kisuk's artwork is characterized by a focu...,simple,"[{'source': '/home/chae/RAG/data-sample.json',...",True


In [57]:
# DataFrame을 CSV 파일로 저장
df.to_csv("dataset_csv/gpt_qa_0213.csv", index=False)

In [52]:
import pandas as pd

df = pd.read_csv('dataset_csv/gpt_qa_0213.csv')

In [58]:
import os
from langchain_teddynote.translate import Translator

# api키 설정
deepl_api_key = os.getenv("DEEPL_API_KEY")

# 객체 생성
translator = Translator(deepl_api_key, "EN", "KO")

# 번역 실행
translated_text = translator("hello, nice to meet you")
print(translated_text)

안녕하세요, 만나서 반가워요


In [59]:
from tqdm import tqdm

# 번역
for i, row in tqdm(df.iterrows(), total=len(df), desc="번역 진행 중"):
    df.loc[i, "question_translated"] = translator(row["question"])
    df.loc[i, "ground_truth_translated"] = translator(row["ground_truth"])

번역 진행 중: 100%|██████████| 10/10 [00:26<00:00,  2.61s/it]


In [60]:
df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done,question_translated,ground_truth_translated
0,\uc6d0\uc2dc\ubbf8\uc220\uc758 \uc601\ud5a5\uc...,"[주제와 색조의 장밋빛 시대(1904–1906)를 지나, 폴 세잔(Paul Céza...",The answer to given question is not present in...,simple,"[{'source': './dataset-sample-1-5.pdf', 'file_...",True,\UC6D0\UC2DC\UBBF8\UC220\UC758 \UC601\UD5A5\UC...,주어진 질문에 대한 답변이 컨텍스트에 없습니다.
1,In what ways do HAN Kisuk's artworks from the ...,[한기석(1930-2011)은 한국의 역사적 원류 속에서 의식적인 뿌리를 찾고 과거...,HAN Kisuk's artworks from the 1960s incorporat...,simple,"[{'source': '/home/chae/RAG/data-sample.json',...",True,1960년대 한기석의 작품에는 실존주의나 추상화와 같은 철학적 주제나 개념이 어떤 ...,1960년대 한기석의 작품은 역사적 자료 속에서 합리적인 질서를 찾고 과거와 현재의...
2,What diverse subjects did Pablo Picasso explor...,[1882-1963)와 공동 창시한 큐비즘(1909–1919)을 통해 20세기의 새...,Pablo Picasso explored diverse subjects in his...,simple,"[{'source': './dataset-sample-1-5.pdf', 'file_...",True,파블로 피카소는 작품에서 어떤 다양한 주제를 탐구했을까요?,"파블로 피카소는 인물 초상화, 삶의 소비, 삶의 비극, 정치적 메시지, 초현실주의의..."
3,What techniques did Pablo Picasso use in his a...,"[• 작품 설명: 파블로 피카소(Pablo Picasso, 1881–1973)는 스...",Pablo Picasso used a variety of techniques in ...,simple,"[{'source': './dataset-sample-1-5.pdf', 'file_...",True,파블로 피카소는 특히 '게르니카'나 '레 드모아젤 드 아비뇽'과 같은 작품에서 새로...,파블로 피카소는 작품에 다양한 기법을 사용하여 새로운 효과를 만들어냈어요. '아비뇽...
4,What distinctive style does Han Kisuk use in h...,[한기석(1930-2011)은 한국의 역사적 원류 속에서 의식적인 뿌리를 찾고 과거...,Han Kisuk uses a distinctive style in his artw...,simple,"[{'source': '/home/chae/RAG/data-sample.json',...",True,한기석 작가는 작품 '팔괘'에서 어떤 독특한 화풍을 사용하나요?,"한기석 작가는 작품 '팔괘'에서 흰색 바탕에 밝은 색을 사용하고 보라색, 노란색 등..."


In [61]:
# question, ground_truth 열을 삭제하고 question_translated, ground_truth_translated 열의 이름을 변경합니다.
df.drop(columns=["question", "ground_truth"], inplace=True)
df.rename(
    columns={
        "question_translated": "question",
        "ground_truth_translated": "ground_truth",
    },
    inplace=True,
)
df.head()

,contexts,evolution_type,metadata,episode_done,question,ground_truth
0,"[주제와 색조의 장밋빛 시대(1904–1906)를 지나, 폴 세잔(Paul Céza...",simple,"[{'source': './dataset-sample-1-5.pdf', 'file_...",True,\UC6D0\UC2DC\UBBF8\UC220\UC758 \UC601\UD5A5\UC...,주어진 질문에 대한 답변이 컨텍스트에 없습니다.
1,[한기석(1930-2011)은 한국의 역사적 원류 속에서 의식적인 뿌리를 찾고 과거...,simple,"[{'source': '/home/chae/RAG/data-sample.json',...",True,1960년대 한기석의 작품에는 실존주의나 추상화와 같은 철학적 주제나 개념이 어떤 ...,1960년대 한기석의 작품은 역사적 자료 속에서 합리적인 질서를 찾고 과거와 현재의...
2,[1882-1963)와 공동 창시한 큐비즘(1909–1919)을 통해 20세기의 새...,simple,"[{'source': './dataset-sample-1-5.pdf', 'file_...",True,파블로 피카소는 작품에서 어떤 다양한 주제를 탐구했을까요?,"파블로 피카소는 인물 초상화, 삶의 소비, 삶의 비극, 정치적 메시지, 초현실주의의..."
3,"[• 작품 설명: 파블로 피카소(Pablo Picasso, 1881–1973)는 스...",simple,"[{'source': './dataset-sample-1-5.pdf', 'file_...",True,파블로 피카소는 특히 '게르니카'나 '레 드모아젤 드 아비뇽'과 같은 작품에서 새로...,파블로 피카소는 작품에 다양한 기법을 사용하여 새로운 효과를 만들어냈어요. '아비뇽...
4,[한기석(1930-2011)은 한국의 역사적 원류 속에서 의식적인 뿌리를 찾고 과거...,simple,"[{'source': '/home/chae/RAG/data-sample.json',...",True,한기석 작가는 작품 '팔괘'에서 어떤 독특한 화풍을 사용하나요?,"한기석 작가는 작품 '팔괘'에서 흰색 바탕에 밝은 색을 사용하고 보라색, 노란색 등..."


In [62]:
# 번역한 데이터셋을 저장합니다.
df.to_csv("dataset_csv/gpt_qa_translate_0213.csv", index=False)

### RAGAS를 활용한 평가

In [63]:
from datasets import Dataset

test_dataset = Dataset.from_pandas(df)
test_dataset

Dataset({
    features: ['contexts', 'evolution_type', 'metadata', 'episode_done', 'question', 'ground_truth'],
    num_rows: 10
})

In [65]:
import ast


# contexts 컬럼의 문자열을 리스트로 변환
def convert_to_list(example):
    contexts = ast.literal_eval(example["contexts"])
    return {"contexts": contexts}

test_dataset = test_dataset.map(convert_to_list)
print(test_dataset)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]


ValueError: malformed node or string: ["주제와 색조의 장밋빛 시대(1904–1906)를 지나, 폴 세잔(Paul Cézanne, 1839-\n1906)과 원시미술의 영향을 받은 〈아비뇽의 여인들(Les Demoiselles d'Avignon)〉\n(1906–1907)을 통해 혁명적 화풍의 시작을 알렸다. 조르주 브라크(Georges Braque,"]

In [21]:
test_dataset[1]["contexts"]

["• 작품 설명: 파블로 피카소(Pablo Picasso, 1881–1973)는 스페인 말라가(Málaga)에서\n태어나 미술 교사인 아버지에게 일곱살까지 미술 교육을 받고, 바르셀로나와\n마드리드 미술학교를 거치며 일찌감치 아카데믹한 화풍으로 뛰어난 기량을 보였다.\n1904년 파리로 건너가 영구 정착하였으며, 무쟁(Mougins)에서 사망하기까지\n프랑스를 주 무대로 활동하면서 전 생애에 걸친 왕성한 창작활동을 펼쳤다. 가난과\n우울, 내면의 고통을 청색의 단색조로 형상화하던 청색 시대(1901–1904)와 감상적인\n주제와 색조의 장밋빛 시대(1904–1906)를 지나, 폴 세잔(Paul Cézanne, 1839-\n1906)과 원시미술의 영향을 받은 〈아비뇽의 여인들(Les Demoiselles d'Avignon)〉\n(1906–1907)을 통해 혁명적 화풍의 시작을 알렸다. 조르주 브라크(Georges Braque,\n1882-1963)와 공동 창시한 큐비즘(1909–1919)을 통해 20세기의 새로운 조형\n언어를 선보였으며 이후 신고전주 의와 초현실주의적 요소를 활용해 방대한 양의\n회화, 조각, 드로잉, 판화, 공예 작품을 제작하였다. 또한 1937년에 제작한\n〈게르니카(Guernica)〉를 통해 역사화를 부활시켰다. 피카소는 다양한 재료의 효과와\n재질감의 대비로부터 나오는 표현성을 보편화시키고, 콜라주나 아상블라주와 같은\n당대 혁신적인 조형 작업을 선보이며 장르 불문의 새로운 시각예술 언어를\n제시하였다. 그는 인물 초상, 사생활의 소산, 생의 희비극, 정치적 메시지, 대담하고\n우아한 에로티시즘 등 다양한 주제를 망라했다.〈얼굴〉은 물병 입구가 향하는 정면\n방향으로 인물의 이목구비를 표현했다. 1940년대부터 말년에 이르기까지 도예\n작업에 몰두했던 피카소가 가장 많이 작업한 도기 중 하나는 물병이었다. 이\n작품에서 피카소는 검은색 선으로 얼굴 윤곽선을 그렸으며, 그 외 바탕 부분에는\n갈색, 검은색, 파란색 화장토를 부분적으로 채

In [22]:
batch_dataset = [question for question in test_dataset["question"]]
batch_dataset[:3]

["파블로 피카소의 예술적 발전에 있어 '레 드모아젤 다비뇽'은 어떤 의미가 있었나요?",
 "파블로 피카소는 특히 '게르니카'와 같이 잘 알려진 작품에서 컬러 레이어링을 작품에 어떻게 사용했을까요?",
 '파블로 피카소의 작품에서 얼굴은 인상이나 이미지 측면에서 어떻게 묘사되어 있나요?']

In [24]:
from langchain_community.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("nlpai-lab/KURE-v1")


# 기존 DB 로드 
persist_directory = "./faiss_artworks_0114_docx"

try:
    faiss_db = FAISS.load_local(
        folder_path=persist_directory,
        embeddings=embedding_model,
        allow_dangerous_deserialization=True  # 신뢰할 수 있는 소스에서만 사용
    )
    
    # embedding_function 수정
    faiss_db.embedding_function = lambda text: (
        embedding_model.encode(text) if isinstance(text, str) else embedding_model.encode(str(text))
    )
    
    print("FAISS 데이터베이스가 성공적으로 로드되었습니다!")
except Exception as e:
    print(f"FAISS 데이터베이스 로드 중 오류 발생: {e}")

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


FAISS 데이터베이스가 성공적으로 로드되었습니다!


In [25]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

import torch
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

# 모델과 토크나이저 로드 (CUDA 사용)
model_id = "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="cuda",  # CUDA에서 자동 배치
    trust_remote_code=True
)


from transformers import pipeline

# 파이프라인 생성
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,  # 생성할 최대 토큰 수 증가
    do_sample=True,        # 샘플링 활성화
    temperature=0.1,      
    top_k=50,             
    repetition_penalty=1.05
)
# LangChain의 HuggingFacePipeline 사용
llm = HuggingFacePipeline(pipeline=pipe)

from langchain.prompts import ChatPromptTemplate

template = '''
<|system|>
You are a friendly chatbot specializing in artworks and general conversations.
Your primary role is to answer questions strictly based on the information provided in the document (context). 
If the requested information is not found in the document, respond with:
"The document does not contain this information." in Korean.

However, if the question is a general conversation or does not relate to the document, you should respond naturally as a conversational chatbot. 
You can talk about art history, artists, exhibitions, and general topics such as daily life, technology, and culture. 
Maintain a friendly and engaging tone, ensuring all responses are written in **Korean**.
Always use **beautiful Markdown formatting** (headings, bullet points, bold or italic text) to enhance readability.

<|context|>
{context}

<|user|>
Question: {question}

<|assistant|>
'''

# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_template(template)


retriever = faiss_db.as_retriever(
    search_kwargs={
        "k": 5,                # 검색 결과 개수
        "fetch_k": 20,         # 더 많은 결과 가져오기
        "mmr": True,           # MMR 활성화
        "mmr_beta": 0.8      # 다양성과 관련성 간 균형
    }
)

import re
class MarkdownOutputParser:
    """Enhanced Markdown parser with additional formatting options."""

    def __call__(self, llm_output):
        # <assistant> 이후의 텍스트만 추출
        match = re.search(r"<\|assistant\|>\s*(.*)", llm_output, re.DOTALL)
        if match:
            extracted_text = match.group(1).strip()
            # 마크다운 코드 블록으로 출력 포맷
            return f"### 모델 결과\n\n{extracted_text}\n\n"
        else:
            # <assistant> 태그가 없는 경우 원래 출력 반환
            return f"### 모델 결과\n\n{llm_output.strip()}\n\n"


from langchain.schema.runnable import RunnablePassthrough, RunnableMap
from langchain_core.output_parsers.string import StrOutputParser
from langchain.prompts import ChatPromptTemplate
chain = (
    RunnableMap({
        "context": retriever,               # Retriever에서 반환된 값을 가져옴
        "question": RunnablePassthrough()   # 질문은 그대로 전달
    })
    | (lambda x: {
        "context": "\n".join([doc.page_content for doc in x["context"]]),
        "question": x["question"]
    })  # context를 문자열로 변환
    | prompt                               # Prompt Template에 전달
    | llm                                  # LLM으로 응답 생성
    | MarkdownOutputParser()                    # 응답을 문자열로 변환
)




Loading checkpoint shards: 100%|██████████| 7/7 [00:25<00:00,  3.64s/it]
Device set to use cuda
/tmp/ipykernel_55283/4033088922.py:40: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  llm = HuggingFacePipeline(pipeline=pipe)


In [26]:
answer = chain.batch(batch_dataset)
answer[:3]

["### 모델 결과\n\n파블로 피카소의 예술적 발전에 있어서 〈아비뇽의 여인들(Les Demoiselles d'Avignon)〉은 획기적인 전환점으로 여겨집니다. 이 작품은 다음과 같은 중요한 의미를 지닙니다:\n\n### 1. **큐비즘의 시작**\n- **혁명적 변화**: 〈아비뇽의 여인들〉은 피카소가 조르주 브라크와 함께 큐비즘을 창시하는 데 결정적인 역할을 했습니다. 이 작품은 전통적인 시각 표현을 벗어나 다각도에서 대상을 동시에 보여주는 혁신적인 접근법을 도입했습니다.\n- **형태의 해체와 재구성**: 인물의 얼굴과 몸이 기하학적인 형태로 분해되고 재구성되어, 기존의 입체감과 공간감을 뛰어넘는 새로운 시각적 경험을 제공합니다.\n\n### 2. **표현의 다양성**\n- **다양한 영감의 융합**: 이 작품은 아프리카 마스크의 영향을 받아, 원시적이고 강렬한 표현력을 도입했습니다. 이는 피카소의 예술 세계에 다양한 영감의 융합을 가져왔습니다.\n- **심리적 깊이**: 작품은 단순히 외형적인 표현을 넘어, 인물들의 심리적 상태와 긴장감을 효과적으로 전달합니다.\n\n### 요약\n〈아비뇽의 여인들〉은 피카소의 예술적 경로에서 전통적인 미술 관습을 탈피하고, 20세기 미술의 새로운 방향을 제시한 획기적인 작품입니다. 이 작품을 통해 피카소는 이후의 다양한 예술적 실험과 혁신을 가능하게 했습니다. 이러한 이유로 이 작품은 피카소의 예술적 발전사에서 매우 중요한 위치를 차지하고 있습니다.\n\n",
 '### 모델 결과\n\n파블로 피카소는 특히 〈게르니카〉와 같은 작품에서 컬러 레이어링을 매우 독특하고 강력한 방식으로 활용했습니다. 하지만 주의할 점은 〈게르니카〉는 주로 흑백으로 제작된 대형 벽화로, 전통적인 의미의 컬러 레이어링보다는 강렬한 흑백 대비와 회색조의 다양한 톤을 통해 감정과 메시지를 전달했습니다. \n\n피카소는 다음과 같은 방법으로 컬러 레이어링의 개념을 적용했습니다:\n\n* **흑백의 힘:** 〈게르니카〉는 흑백으로 그려져 있지만

In [27]:
# 'answer' 컬럼 덮어쓰기 또는 추가
if "answer" in test_dataset.column_names:
    test_dataset = test_dataset.remove_columns(["answer"]).add_column("answer", answer)
else:
    test_dataset = test_dataset.add_column("answer", answer)

### 답변 평가

In [28]:
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset=test_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

result

Evaluating: 100%|██████████| 40/40 [00:39<00:00,  1.01it/s]


{'context_precision': 0.8000, 'faithfulness': 0.3073, 'answer_relevancy': 0.0776, 'context_recall': 0.2500}

In [29]:
result_df = result.to_pandas()
result_df.head()

,question,contexts,answer,ground_truth,context_precision,faithfulness,answer_relevancy,context_recall
0,파블로 피카소의 예술적 발전에 있어 '레 드모아젤 다비뇽'은 어떤 의미가 있었나요?,"[• 작품 설명: 파블로 피카소(Pablo Picasso, 1881–1973)는 스...",### 모델 결과\n\n파블로 피카소의 예술적 발전에 있어서 〈아비뇽의 여인들(Le...,'레 드모아젤 다비뇽'은 폴 세잔과 원시 미술의 영향을 받은 파블로 피카소의 예술적...,1.0,1.000000,0.000000,0.0
1,파블로 피카소는 특히 '게르니카'와 같이 잘 알려진 작품에서 컬러 레이어링을 작품에...,"[• 작품 설명: 파블로 피카소(Pablo Picasso, 1881–1973)는 스...",### 모델 결과\n\n파블로 피카소는 특히 〈게르니카〉와 같은 작품에서 컬러 레이...,주어진 질문에 대한 답변이 컨텍스트에 없습니다.,1.0,0.000000,0.000000,0.0
2,파블로 피카소의 작품에서 얼굴은 인상이나 이미지 측면에서 어떻게 묘사되어 있나요?,[• 작품 번호: 10288\n• 제작 연도: 1969\n• 크기: 22×22×2\...,### 모델 결과\n\n파블로 피카소의 작품에서 얼굴은 다양한 방식으로 묘사되어 인...,"파블로 피카소의 작품 속 얼굴은 얼굴 중앙에 직사각형 패널이 있고, 살짝 미소를 지...",1.0,0.000000,0.000000,0.0
3,파블로 피카소의 작품 '게르니카'는 어떤 역사적 의미를 지니고 있나요?,[• 작품 번호: 10288\n• 제작 연도: 1969\n• 크기: 22×22×2\...,### 모델 결과\n\n파블로 피카소의 작품 〈게르니카(Guernica)〉는 역사적...,파블로 피카소의 작품 '게르니카'는 역사적 사건을 묘사하는 데 사용된 작품으로 역사...,1.0,0.555556,0.000000,1.0
4,피카소는 작품에서 어떻게 혁신적인 예술적 표현을 사용했을까요?,[〈게르니카(Guernica)〉를 통해 역사화를 부활시켰다. 피카소는 다양한 재료의...,### 모델 결과\n\n피카소는 다양한 방식으로 혁신적인 예술적 표현을 작품에 녹여...,피카소는 다양한 재료와 기법을 사용하고 형태를 단순화하며 새로운 관점을 탐구하는 등...,1.0,0.041667,0.775519,0.5


In [31]:
result_df.to_csv("dataset_csv/ragas_result_0213.csv", index=False)

In [32]:
result_df.loc[:, "context_precision":"context_recall"]

,context_precision,faithfulness,answer_relevancy,context_recall
0,1.0,1.000000,0.000000,0.0
1,1.0,0.000000,0.000000,0.0
2,1.0,0.000000,0.000000,0.0
3,1.0,0.555556,0.000000,1.0
4,1.0,0.041667,0.775519,0.5
5,0.0,0.200000,0.000000,0.0
6,0.0,0.250000,0.000000,0.0
7,1.0,0.000000,0.000000,0.0
8,1.0,0.454545,0.000000,0.0
9,1.0,0.571429,0.000000,1.0
